# TP5

Elaborado por: Alan Churichi

## Objetivo: 

- Implementar el detector de fondo naive usando la mediana como estimador. El algoritmo debe recibir el parámetro N (cantidad de frames utilizados para la estimación) y el intervalo de tiempo para recalcular el fondo.
- Se deben generar las mascaras de foreground y aplicarlas a los frames para segmentar los objetos en movimiento.
- Comparar con alguno de los métodos vistos en la practica basados en mezcla de gaussianas.

In [1]:
%load_ext lab_black

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import time

In [2]:
base_path = "/tf/notebooks/CEIA/computer-vision-1/tp5/assets"
video_path = os.path.join(base_path, "vtest.avi")

Lo primero que se hace es generar un conjunto de funciones que nos serán de utilidad en nuestra función para remover el fondo.

- `get_random_frames`: toma una muestra aleatoria de n frames en el rango especificado.
- `get_n_frames`: toma una muestra de n frames consecutivos.
- `write_filename`: toma una frame y le aplica un recuadro con el texto especificado.

In [3]:
def get_random_frames(cap, n_samples, start, end):
    start_frame = cap.get(cv.CAP_PROP_POS_FRAMES)
    random_indices = [random.randint(start, end) for _ in range(n_samples)]
    frames = []

    for index in random_indices:
        cap.set(cv.CAP_PROP_POS_FRAMES, index)
        frames.append(cap.read()[1])

    cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
    return np.array(frames)


def get_n_frames(cap, n_samples):
    start_frame = cap.get(cv.CAP_PROP_POS_FRAMES)
    frames = []

    for i in range(n_samples):
        cap.set(cv.CAP_PROP_POS_FRAMES, start_frame + i)
        frames.append(cap.read()[1])

    cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
    return np.array(frames)


def write_filename(frame, filename):
    cv.rectangle(frame, (10, 2), (300, 20), (255, 255, 255), -1)
    cv.putText(
        frame,
        filename,
        (15, 15),
        cv.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 0, 0),
    )
    return frame

Definimos la función principar `remove_bg`, el funcionamiento es el siguiente:

- Se carga el video desde el path especifcado.
- Se calculan características del video como el numero total de frames, los fps y la cantidad de frames para actualizar la mediana de comparación.
- Se itera por steps, que son los intervalos en los que se debe calcular la mediana. En cada step se calcula la mediana, si es la primera iteración se toman los n primeros frames, sino se toman n frames aleatorios en el rango del step anterior.
- En cada step se itera sobre todos los frames dentro de ese step y se calcula el fondo haciendo la resta del `frame` - `mediana` en escala de grises.
- Se binariza el resultado y, de manera opcional, se hace una operación morfológica de apertura.
- Se utiliza `VideoWriter` para guardar el stream frame a frame.

In [4]:
def remove_bg(video_path, dst_path, bg_samples, recalculation, apply_morph=True):
    """
    Remueve el backgound de un video
    Argumentos:
        video_path: Path del video a procesar
        dst_path: Path donde guardar la mascara
        bg_samples: El numero de muestras a tomar para calcular el background
        recalculation: Intervalo en segundos para recalcular la mediana
        apply_morph: Si se desea o no aplicar apertura al resultado
    """
    start_time = time.time()

    cap = cv.VideoCapture(video_path)
    total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv.CAP_PROP_FPS))
    width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

    video_writer = cv.VideoWriter(dst_path, 0, fps, (width, height))

    step = recalculation * fps

    for lower_limit in range(0, total_frames, step):
        upper_limit = min(lower_limit + step, total_frames)
        progress = int(lower_limit / total_frames * 100)
        print(f"Generando video {progress}%")

        if lower_limit < step:
            random_frames = get_n_frames(cap, bg_samples)
        else:
            random_frames = get_random_frames(
                cap, bg_samples, lower_limit - step, lower_limit
            )

        median = np.median(random_frames, axis=0)
        median_gray = cv.cvtColor(median.astype(np.uint8), cv.COLOR_BGR2GRAY)

        for i in range(lower_limit, upper_limit):
            frame = cap.read()[1]
            frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            diff = cv.absdiff(frame_gray, median_gray)
            _, result = cv.threshold(diff, 30, 255, cv.THRESH_BINARY)
            if apply_morph:
                result = cv.morphologyEx(
                    result, cv.MORPH_OPEN, np.ones((3, 3), np.uint8)
                )
            result = write_filename(result, os.path.basename(dst_path))

            video_writer.write(cv.cvtColor(result, cv.COLOR_GRAY2BGR))

    elapsed_time = time.time() - start_time

    print(f"Generando video 100%")
    print(f"Tiempo empleado: {elapsed_time:.3f} segundos")

    cv.destroyAllWindows()
    cap.release()
    video_writer.release()

Probamos el algoritmo

In [5]:
dst_path = os.path.join(base_path, "output-median.avi")
remove_bg(video_path, dst_path, 60, 10)

Generando video 0%
Generando video 12%
Generando video 25%
Generando video 37%
Generando video 50%
Generando video 62%
Generando video 75%
Generando video 88%
Generando video 100%
Tiempo empleado: 35.352 segundos


Se obtuvo el siguente resultado:

<img src="assets/example-output-median.png" width="800" align="center">

Probamos el mismo algorimto pero esta vez sin aplicar apertura

In [6]:
dst_path = os.path.join(base_path, "output-median-no-morph.avi")
remove_bg(video_path, dst_path, 60, 10, apply_morph=False)

Generando video 0%
Generando video 12%
Generando video 25%
Generando video 37%
Generando video 50%
Generando video 62%
Generando video 75%
Generando video 88%
Generando video 100%
Tiempo empleado: 35.095 segundos


Se obtuvo el siguente resultado:

<img src="assets/example-output-median-no-morph.png" width="800" align="center">

Definimos una nueva función que utiliza los algoritmos de eliminación de background provistos por OpenCV.

In [7]:
def open_cv_remove_bg(video_path, dst_path, method="MOG2"):
    """
    Remueve el backgound de un video utilizando algoritmos de OpenCV
    Argumentos:
        video_path: Path del video a procesar
        dst_path: Path donde guardar la mascara
        method: Algoritmo de OpenCV a utilizar
    """

    if method == "MOG2":
        bg_sub = cv.createBackgroundSubtractorMOG2()
    elif method == "KNN":
        bg_sub = cv.createBackgroundSubtractorKNN()
    else:
        return

    start_time = time.time()

    cap = cv.VideoCapture(video_path)
    width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv.CAP_PROP_FPS))
    video_writer = cv.VideoWriter(dst_path, 0, fps, (width, height))

    while True:
        _, frame = cap.read()

        if frame is None:
            break

        mask = bg_sub.apply(frame)
        result = write_filename(mask, os.path.basename(dst_path))

        video_writer.write(cv.cvtColor(result, cv.COLOR_GRAY2BGR))

    elapsed_time = time.time() - start_time

    print(f"Tiempo empleado: {elapsed_time:.3f} segundos")

    cv.destroyAllWindows()
    cap.release()
    video_writer.release()

In [8]:
dst_path = os.path.join(base_path, "opencv-MOG2.avi")
open_cv_remove_bg(video_path, dst_path, method="MOG2")

Tiempo empleado: 5.723 segundos


Se obtuvo el siguente resultado:

<img src="assets/example-opencv-MOG2.png" width="800" align="center">

In [9]:
dst_path = os.path.join(base_path, "opencv-KNN.avi")
open_cv_remove_bg(video_path, dst_path, method="KNN")

Tiempo empleado: 5.170 segundos


Se obtuvo el siguente resultado:

<img src="assets/example-opencv-KNN.png" width="800" align="center">

## Conclusiones

En base a las pruebas realizadas se obtuvieron la siguentes conclusiones:

- Todos los algoritmos dan resutlados bastante buenos a la hora de separar el fondo de los objetos en movimiento.
- Los métodos de OpenCV son aproximadamente 7 veces más rápido que el que algoritmo de mediana que se implementó.
- El algoritmo implementado tiene menos ruido que los métodos de OpenCV.
- Utilizar apertura no hace el algoritmo mucho más lento. Aproximadamente 200ms más.
- Utilizar apertura elimina en cierta medida el ruido pero también deforma la máscara.
- En los videos del algoritmo implementado se puede apreciar cierto ruido, de imagénes estáticas, producto de los tiempos de actualización del filtro de mediana.
- La forma en la que se codifica el video no es la más óptima, ya que los archivos de salida tienen un peso de 527Mb. Siendo que el archivo de lectura pesa solamente 8Mb.

Se puede ver el video con la comparativa de las 4 pruebas realizadas en el siguiente [link](https://youtu.be/eNfNtkFaRwY)